In [1]:
import os
import datetime
import uuid
from tqdm import tqdm

import torch
import torch.nn    as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from data_declaration import Task
from loader_helper    import LoaderHelper


from vapformer.model_components import thenet,UnetrPP
from evaluation import evaluate_model
import matplotlib.pyplot as plt
import numpy as np
import random
from torchmetrics.classification import BinaryAUROC
from vapformer.dynunet_block import get_conv_layer, UnetResBlock
from monai.networks.layers.utils import get_norm_layer

from sklearn import neighbors
import scipy.sparse as sp

/home/ubuntu22/anaconda3/envs/VAPFormer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device("cuda:0")

ld_helper = LoaderHelper(task=Task.NC_v_AD)
train_dl = ld_helper.get_train_dl(0, batch_size = 8)
test_dl = ld_helper.get_test_dl(0, batch_size = 16)

In [3]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

__all__ = ['ResNet3D', 'resnet3d18']

model_urls = {
    'resnet3d18': 'https://download.pytorch.org/models/resnet3d18-5c106cde.pth',
}

def conv3x3x3(in_planes, out_planes, stride=1):
    """3x3x3 convolution with padding"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv1x1x1(in_planes, out_planes, stride=1):
    """1x1x1 convolution"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet3D(nn.Module):

    def __init__(self, block, layers, num_classes=1, zero_init_residual=False):
        super(ResNet3D, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv3d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        # Remove the final fully connected layer and average pooling layer
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.sig = nn.Sigmoid()
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm3d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Remove the final average pooling and fully connected layer
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.sig(x)

        return x

def _resnet3d(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet3D(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

def resnet3d18(pretrained=False, progress=True, **kwargs):
    return _resnet3d('resnet3d18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                     **kwargs)

resnet18 = resnet3d18(pretrained=False)


In [4]:
def evaluate(model_in, test_dl, thresh=0.5, param_count=False):
        
    correct = 0; total = 0
    model_in.eval()
    total_label = torch.tensor([]).to(DEVICE)
    total_pre = torch.tensor([]).to(DEVICE)
    
    TP = 0.000001; TN = 0.000001; FP = 0.000001; FN = 0.000001
    
    with torch.no_grad():
        
        for i_batch, sample_batched in enumerate(test_dl):
            
            batch_X  = sample_batched['mri'].to(DEVICE).float()
            batch_y  = sample_batched['label'].to(DEVICE).float()

            net_out = model_in(batch_X)
            total_label = torch.cat((total_label,batch_y),1)
            total_pre = torch.cat((total_pre,net_out),1)


            for i in range(len(batch_X)):
                
                real_class = batch_y[i].item()

                # print("real class: "+real_class)
                # print("out class: "+net_out[i])
                predicted_class = 1 if net_out[i] > thresh else 0      
                
                if (predicted_class == real_class):
                    correct += 1
                    if (real_class == 0):
                        TN += 1
                    elif (real_class == 1):
                        TP += 1
                else:
                    if (real_class == 0):
                        FP += 1
                    elif (real_class == 1):
                        FN += 1
                    
                    
                total += 1

    metric = BinaryAUROC(thresholds=None)
    auc = metric(total_pre, total_label).item()
    
    sensitivity = round((TP / (TP + FN)), 5)
    specificity = round((TN / (TN + FP)), 5)
    accuracy = round((sensitivity+specificity)/2, 5)
    
    return accuracy, sensitivity, specificity, auc


In [5]:
# 0.80 2mm
# 0.80 1.5mm
resnet18.to(DEVICE)
optimizer = optim.AdamW(resnet18.parameters(), lr=0.00001, weight_decay=5e-4)


loss_function = nn.BCELoss()
loss_fig = []
eva_fig = []

epochs = 100
best_auc = 0
nb_batch = len(train_dl)



# Train
for i in range(1,1+epochs):
    loss = 0.0
    resnet18.train()
    for _, sample_batched in enumerate(tqdm(train_dl)):

        batch_x = sample_batched['mri'].to(DEVICE).float()
        batch_y = sample_batched['label'].to(DEVICE).float()

        optimizer.zero_grad()
        outputs = resnet18(batch_x)
        
        batch_loss = loss_function(outputs, batch_y)
        
        batch_loss.backward()
        optimizer.step()


        loss += float(batch_loss) / nb_batch

    tqdm.write("Epoch: {}/{}, train loss: {}".format(i, epochs, round(loss, 5)))
    # filein.write("Epoch: {}/{}, train loss: {}\n".format(i, epochs, round(loss, 5)))
    loss_fig.append(round(loss, 5))
    accuracy, sensitivity, specificity, auc = evaluate(resnet18, test_dl)
    eva_fig.append(accuracy)
    tqdm.write("Epoch: {}/{}, evaluation loss: {}".format(i, epochs,(accuracy, sensitivity, specificity, auc)))
    # filein.write("Epoch: {}/{}, evaluation loss: {}\n".format(i, epochs,(accuracy, sensitivity, specificity, auc)))




100%|███████████████████████████████████████████████████████████████████████████| 43/43 [00:09<00:00,  4.54it/s]

Epoch: 1/100, train loss: 0.69962


Epoch: 1/100, evaluation loss: (0.5, 0.0, 1.0, 0.5267750024795532)


100%|███████████████████████████████████████████████████████████████████████████| 43/43 [00:08<00:00,  5.14it/s]

Epoch: 2/100, train loss: 0.60385


...

Epoch: 99/100, train loss: 0.00541


Epoch: 99/100, evaluation loss: (0.66482, 0.35789, 0.97175, 0.8898602724075317)


100%|███████████████████████████████████████████████████████████████████████████| 43/43 [00:08<00:00,  5.26it/s]

Epoch: 100/100, train loss: 0.0056


Epoch: 100/100, evaluation loss: (0.64094, 0.31579, 0.9661, 0.8923580646514893)


In [ ]:
all_outputs = []
# 将模型设置为评估模式
model.eval()
for _, sample_batched in enumerate(tqdm(train_dl)):
    batch_x = sample_batched['mri'].to(DEVICE).float()
    with torch.no_grad():
        outputs = model(batch_x)
    
    all_outputs.append(outputs)
    stacked_outputs = torch.cat(all_outputs, dim=0)
    

In [10]:
stacked_outputs.shape

torch.Size([350, 512])

In [12]:
def create_graph_from_embedding(embedding, name, n = 30):
    latent_dim, batch_size = embedding.shape
    if name == 'knn':
        A = neighbors.kneighbors_graph(embedding, n_neighbors = n).toarray()
        A = (A + np.transpose(A)) / 2
        return A

In [14]:
stacked_outputs = stacked_outputs.cpu()
adj = create_graph_from_embedding(stacked_outputs, name='knn', n=30)

In [16]:
adj.shape

(350, 350)

In [17]:
from utils import *

In [18]:
adj, edge = preprocess_adj(adj)
adj = adj.todense()

In [21]:
class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim, use_bias=True):
        super(GraphConvolution, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.use_bias = use_bias
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        if self.use_bias:
            self.bias = nn.Parameter(torch.Tensor(output_dim))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight)
        if self.use_bias:
            init.zeros_(self.bias)

    def forward(self, input_feature, adjacency):

        support = torch.mm(input_feature, self.weight)
        output = torch.mm(adjacency, support)
        if self.use_bias:
            output += self.bias
        return output

In [ ]:
class SYNet_AD(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SYNet_AD, self).__init__()
        nd_1 = 128
        nd_2 = 64
        nd_3 = 32
        n_4 = 2

        self.gcn1_1 = GraphConvolution(input_dim, nd_1)  # AD-NC
        self.gcn1_2 = GraphConvolution(nd_1, nd_2)


        self.linear1 = torch.nn.Linear(93, n_4)  # AD-NC
        self.linear2 = torch.nn.Linear(n_4, output_dim)
        self.linear3 = torch.nn.Linear(output_dim, output_dim)




    def forward(self, adjacency1, feature, adjacency2):
        # adjacency1:S  adjacency2:A feature:X
        lam1 = 1e-5
        lam2 = 1e-3
        y_hat = F.relu(self.gcn1_1(feature.t(), adjacency1))
        y_hat = F.dropout(y_hat, 0.2, training=self.training)
        y_hat = self.gcn1_2(y_hat, adjacency1)
        
        h = self.linear1(y_hat)
        h = F.relu(self.linear2(h))
        h = self.linear3(h)

        logits = h

        return logits, X_a, X_s

In [22]:
def evaluate(model_in, test_dl, thresh=0.5, param_count=False):
        
    correct = 0; total = 0
    model_in.eval()
    total_label = torch.tensor([]).to(DEVICE)
    total_pre = torch.tensor([]).to(DEVICE)
    
    TP = 0.000001; TN = 0.000001; FP = 0.000001; FN = 0.000001
    
    with torch.no_grad():
        
        for i_batch, sample_batched in enumerate(test_dl):
            
            batch_X  = sample_batched['mri'].to(DEVICE)
            batch_clinical = sample_batched['clin_t'].to(DEVICE)
            batch_y  = sample_batched['label'].to(DEVICE)

            net_out = model_in(batch_X,batch_clinical)
            total_label = torch.cat((total_label,batch_y),1)
            total_pre = torch.cat((total_pre,net_out),1)


            for i in range(len(batch_X)):
                
                real_class = batch_y[i].item()

                # print("real class: "+real_class)
                # print("out class: "+net_out[i])
                predicted_class = 1 if net_out[i] > thresh else 0      
                
                if (predicted_class == real_class):
                    correct += 1
                    if (real_class == 0):
                        TN += 1
                    elif (real_class == 1):
                        TP += 1
                else:
                    if (real_class == 0):
                        FP += 1
                    elif (real_class == 1):
                        FN += 1
                    
                    
                total += 1

    metric = BinaryAUROC(thresholds=None)
    auc = metric(total_pre, total_label).item()
    
    sensitivity = round((TP / (TP + FN)), 5)
    specificity = round((TN / (TN + FP)), 5)
    accuracy = round((sensitivity+specificity)/2, 5)


In [31]:
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)


loss_function = nn.BCELoss()
loss_fig = []
eva_fig = []

epochs = 100
best_auc = 0
nb_batch = len(train_dl)



# Train
for i in range(1,1+epochs):
    loss = 0.0
    model.train()
    for _, sample_batched in enumerate(tqdm(train_dl)):

        batch_x = sample_batched['mri'].to(DEVICE).float()
        batch_y = sample_batched['label'].to(DEVICE)

        model.zero_grad()
        outputs = model(batch_x)
        
        batch_loss = loss_function(outputs, batch_y)
        
        batch_loss.backward()
        optimizer.step()


        loss += float(batch_loss) / nb_batch

    tqdm.write("Epoch: {}/{}, train loss: {}".format(i, epochs, round(loss, 5)))
    # filein.write("Epoch: {}/{}, train loss: {}\n".format(i, epochs, round(loss, 5)))
    loss_fig.append(round(loss, 5))
    accuracy, sensitivity, specificity, auc = evaluate(model_in, test_dl)
    eva_fig.append(accuracy)
    tqdm.write("Epoch: {}/{}, evaluation loss: {}".format(i, epochs,(accuracy, sensitivity, specificity, auc)))
    # filein.write("Epoch: {}/{}, evaluation loss: {}\n".format(i, epochs,(accuracy, sensitivity, specificity, auc)))




  0%|                                                                                   | 0/175 [00:00<?, ?it/s]


ValueError: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 512])) is deprecated. Please ensure they have the same size.